**Goal: walk through Hidden Markov Model (forward pass) algebra and its link in to the Viterbi optimal path, to derive a quadratic programming formulation of the Viterbi solution.  From here the quadratic program will be converted to a linear program**

Solving for most likely Viterbi optimal path (and associated 'score') via Quadratic Programming and/or Linear Programming is a useful and quite general way to check implementations of Viterbi -- by just following where the algebra takes you.  

note: certain conventions used in the algebra are done for illustrative purposes --i.e. to make the underlying points as clear as possible.  Hence there are a small few adjustments needed to map from the algebra to the Julia code.

The associated code was done in Julia 0.4.5.

There are a lot of moving parts in this post. **For simplicity, this post virtually ignores differences between real valued outputs and integer constraints.** That is, I sometimes act as if we are solving for indicator variables $\in \{0,1\}$ and other times for indicator variables $\in [0,1]$.  **The reason is that this is expedient.**  The fact is we can toggle either interpretation we want as we work through the math.  The only assumption I make is that there is one unique Viterbi optimal path.  **At the *very* end of this post I address the issue of uniquenes and real values vs. Boolean Valued solutions, under the bold heading of "A Final Note".**

as in past posts, let:

$h = \begin{bmatrix}
1\\ 
1\\ 
\vdots\\ 
1
\end{bmatrix}
$
 
What we have is an equation like  
$Posterior_f \propto D_{y_{n-1}} A  ...  D_{y_1}  A D_{y_0}Ax$

This post assumes that there is a valid posterior with some probability > 0 in it.  

I re-label things a bit -- as we are transitioning away from forward-backward to Viterbi and have different goals in mind:  
$EndingProbability \propto (D_{y_{n-1}}  A)  ...  (D_{y_1}  A)  (D_{y_0}A) starting$

For notational simplicity let $B_{i} = (D_i A)$  
$EndingProbability \propto (B_{y_{n-1}})  ...  (B_{y_1})  (B_{y_0}) starting$

Let's simplify this a bit and consider only 4 or so transitions  
$EndingProbability \propto B_{y_{3}} B_{y_{2}}  B_{y_1}  B_{y_0} starting$

So For Viterbi, what we want is is the best path that leads through all of this:  
$MAP_{ending}Probability \propto grab(B_{y_{3}}) * grab(B_{y_{2}}) * grab( B_{y_1}) * grab(B_{y_0}) * grab(starting)$

Where I've introduced --for thought experiment purposes-- a function 'grab' that grabs the best (scalar) choice from each $B_i$ *and* makes sure that there is a coherent chain from starting to ending.  

Now once we have our head around the above thought experiement, we can switch to a negative log space equivalent  
$-Log(MAP_{ending}Probability)  \propto grab( -Log(B_{y_{3}}))+ grab( -Log(B_{y_{2}})) + grab( -Log( B_{y_1})) + grab(-Log(B_{y_0})) + grab(-Log(starting))$

As a reminder, in this negative logspace equivalent world, we have gone from multiplying 'best' probabilities on a chain, to adding their negative logs.  Since the best chain must have probability > 0, no single best probability can be = 0.  Thus we know that all $BestProbabilities > 0$ and all $BestProbabilities \leq 1$ -- in logspace this means all value are $ \leq 0$, and thus in the negative logspace world, we have all values $ \geq 0$ -- i.e. the negative logspace adjustment allows us to make everything additive, and strictly non-negative, which is quite nice.


*The above notation is cumbersome: here are a few more transformations so we can abstract and focus on the important stuff*

1. I am going to drop the $\propto$ sign here for minor semantic reasons.  If it makes you uncomfortable, you can write it back in.  The idea, though, is for a given legal setup, in our problem, there will be one best / MAP / viterbi optimal path score, and that is what we are calculating here.  Note: there are some subtleties around the case where multiple paths have the same MAP score (i.e. non-unique optimal path).  I put some comments in the code to discuss some of this.  This post assumes that there is a single optimal path associated with MAP score.  

2. Let $-Log(B_{y_{i}}) = C_i$  That is $C_i$ means that ith appropriate negative log transformed matrix.  Once you are comfortable with this point (and once it's been coded), you can just think of it as some matrix $C_i$.  You don't need to worry where it came from.

3. I will relabel the -Log(starting) to NominalStartVec.  This helps unclutter our thinking.  (And once the transformation has been coded, you don't need to worry about where it came from.)

4. Also: forget about the $x$ and $y$ earlier in this post! I will be using them again with unrelated meaning!  It's easy to run out of coherent letters in an alphabet when using linear algebra.  So please wipe the prior usage of $x$ and $y$ from your memory!

So what we have here is:

$_{LogWorld}MAP_{ending}Probability = grab( C_{3})+ grab( C_{2}) + grab(C_{1}) + grab(C_{0}) + grab(NominalStartVec))$

Now we have two things to do.  

First replace this all powerful grab function with something mathematically tractable. That yields a quadratic program.  

The optional second step will be to decompose the quadratic program in a particular way so that it becomes linear in its objective function -- much subtler and perhaps harder to follow.


**Step one: convert grab into something mathematically tractable**

To hopefully make this simple, let's say there are 3 possible states at any given time.  (One oddity: by convention, I start counting at zero, as many programming languages do.  The accompanying code is in Julia.  In addition to slightly different variable designations that I use, I'd highlight that Julia *starts counting at one*.  So there will be some gaps between the Julia setup and the algebraic representation.  These gaps should are minor.)

$v = \begin{bmatrix}
v_{0}\\ 
v_{1}\\ 
v_{2}
\end{bmatrix}
$ $w = \begin{bmatrix}
w_{0}\\ 
w_{1}\\ 
w_{2}
\end{bmatrix}
$ $x = \begin{bmatrix}
x_{0}\\ 
x_{1}\\ 
x_{2}
\end{bmatrix}
$ $y = \begin{bmatrix}
y_{0}\\ 
y_{1}\\ 
y_{2}
\end{bmatrix}
$ $z = \begin{bmatrix}
z_{0}\\ 
z_{1}\\ 
z_{2}
\end{bmatrix}
$

For the avoidance of doubt: **each scalar item in a given vector is an indicator variable**.  That is: it takes on a value of 0 or 1.  There is also a very natural constraint that **the sum of any given vector should equal 1**.  That is you can have one indicator variable 'chosen' and then rest are all zero'd out. The fact that the sum of scalar values in any given vector equals one means that, in effect, we *could* think of each vector being like a probability distribution.  This is not needed for the quadratic programming formulation, but becomes quite useful when thinking about how to convert the QP into an LP.

So I've introduced the above vectors, and we end up with the below equation, in quadratic form.  

$_{LogWorld}MAP_{ending}Probability = z^T C_{3} y + y^T C_{2} x  + x^T C_{1} w + w^T C_{0}v + v^T NominalStartVec$



If you understand the above, you can see its very clear links to a (hidden) markov chain model.  We are selecting only one coherent path on the chain, and because we did a negative log space transformation, we can add up all of these 'weights' on the chain.  This is the simplest form of a quadratic program: we have an objective function (i.e. minimize $_{LogWorld}MAP_{ending}Probability $ but all of the constraints are linear in nature).

- - - 
*interlude*
If you'd like, you can 'open up' the $C_i$'s and convert them (and NominalStartVec) back from negative log space to 'regular probability' space.  It's important that we don't change $v, w, x, y, z$ -- they are merely indicator variables being used instead of the 'grab' function.  Being in logspace or not has no impact on them.

$EndingProbability \propto (z^T(D_{y_{3}}A)y) * (y^T(D_{2}A)x) * (x^T(D_{y_1}A)w) * (w^T(D_{y_0}A)v) * (v^T starting)$

The above is a sequence of scalar multiplications: i.e. we have:

$(someScalar)*(AnotherScalar)*(yetAnotherscalar)*(oneMoreScalar)*(aFinalScalar)$.  

So what you see above is quite literally the product of a bunch of different probabilities.  I.e. we've described an arbitrary single pass through a hidden markov model.  In Viterbi we do this while trying to maximize the value of said product, *or* we go to negative log space and minimize the value of this sum. From a Linear or Quadratic Programming standpoint, an equation like 

$EndingProbability \propto (z^T(D_{y_{3}}A)y) * (y^T(D_{2}A)x) * (x^T(D_{y_1}A)w) * (w^T(D_{y_0}A)v) * (v^T starting)$

is not at all useful.  The negative logspace equivalent, however, is very useful.  It still can be nice to think about the above equation though.  It is nice in how readable it is, it can directly be drawn in terms of a trellis diagram, etc. and at its core, the above equation is the generator of this entire post.  

*end interlude*
- - - 


Note: that the domain for any scalar value in vectors $v, w, x, y, z$ is [0, 1].  Also note that $C_i$ has strictly nonnegative values.  So *in effect* we have a quadratic optimization problem that is convex.  (I say *in effect* here, in the same way that I'd say for  single variable function $f(a) = a^3$ is convex if we insist that $a$ is non-negative.) 

Solvers for constrained convex quadratic problems are quite effective.  That said they are still fairly heavy duty machinery, and I noticed some very odd behavior Julia with respect to the solver I used.  E.g. when I put in redundant dummy constraints like that the sum of all elements in those 5 vectors $v, w, x, y, z$ in aggregate is less than 6 -- this somehow changes the answer to something suboptimal.  Clearly each has a max value of 1, and there are 5 of them -- so max value is 5 and the constraint should not impact things.  When I played around with things like this, I would notice changes in the solution from time to time -- and they should not have changed at all.  It is certainly possible that I could have eliminated this odd behavior by using another solver, or tweaking the formulation somewhat.

But the point remains: a Quadratic Program is a heavier duty piece of machinery, and is harder to interpret than a Linear Program.


**The above will yield a working quadratic program. Associated code for such a program is shown in the below.**

Again, the essence of the objective function for this QP formulation is:
$_{LogWorld}MAP_{ending}Probability = z^T C_{3} y + y^T C_{2} x  + x^T C_{1} w + w^T C_{0}v + v^T NominalStartVec$

In [2]:
function quadratic_program_objective_function(likelihood_matrix, A, y_vec, x_box, nominal_prior_x)
    """
    The objective function for the QP formulation.  
    
    NOTE THIS IS SETUP FOR THE VITERBI COIN EXAMPLE, 
    hence there is some special handling around time 1
    
    This approach in essence, should work, though it is something of a hack as 
    it perturbs A (and the likelihood function) ever so slightly, and blows up associated sparsity
    
    Getting the quadratic program to properly take advantage of sparsity is challenging.  
    As noted at the very end of this writeup, taking advantage of sparsity 
    in the LP formulation is much easier, though that is as of yet and unimplemented optimization.  
    
    The small perturbation of all cells by epsilon is the easiest way
    to (approximately) solve the problem being posed and keep the essence of the problem most clear
    """
    epsilon = 0.000000000000000001
    # interesting side note: we could vary the epsilon by a TINY amount, at random 
    
    assert(length(size(y_vec)) == 1) # a proper vector here
    entries_in_y = size(y_vec)[1] # then we grab its dimension here
        
    collector = dot(nominal_prior_x , x_box[:,1])
    # recall nominal_prior_x is already in - log space

    for idx = 1:entries_in_y        
        # the_diagonal_piece = diagm(likelihood_matrix[:,y_vec[idx]]) 
        # not needed if we use broadcasting
        if idx > 1
            adj_A = ( likelihood_matrix[:,y_vec[idx]] .* A) + epsilon
            # below is the correct linear algebra setup, but above is the broadcasting equivalent
            # adj_A = *(the_diagonal_piece, A) + epsilon            
        else
            # special handling needed for the first toss in the Viterbi coin example from  6.008.1x
            the_diagonal_piece = diagm(likelihood_matrix[:,y_vec[idx]])
            adj_A = the_diagonal_piece + epsilon
        end        
        adj_A = -log(adj_A)
                
        new_vec = *(adj_A, x_box[:, idx])
        
        collector += dot(x_box[:, idx + 1], new_vec)        

    end
    return collector
end


quadratic_program_objective_function (generic function with 1 method)

In [3]:
# this is the quadratic setup.  
# encapsulating this in a function would likely lead to better performance, 
# though the 'open' setup here allows the user to more easily explore results

using JuMP
using Ipopt
mymodel = Model(solver=IpoptSolver(print_level=0))

# For the avoidance of doubt:
# A is the transition matrix, and L is a collection of likelihood vectors

A = [3/4  1/4; 
     1/4  3/4]

L = [1/4 1/2  1/2; 
     1/8 1/4  3/4]

assert(size(A)[1]== size(A)[2]) # that A is square
r_states = size(A)[1]

y_vec = [2, 3, 3, 3]
nominal_prior_x = [1/4, 1/8]

nominal_prior_x = nominal_prior_x / sum(nominal_prior_x)
nominal_prior_x = -log(nominal_prior_x)
# convert to logspace before beginning 

n_updates = size(y_vec)[1]

@variable(mymodel, 0<=x[i = 1:r_states*(n_updates +1)]<=1)
x_box = reshape(x, (r_states, (n_updates + 1)  ))
# reshape x into a matrix called x_box.  This makes handling easier

ones_v_days_as_row_vec = ones(1,r_states)

# linear equality constraint
my_constraint = @constraint(mymodel, *(ones_v_days_as_row_vec, x_box) .== 1) 

# quadratic objective function
z = quadratic_program_objective_function(L, A, y_vec, x_box, nominal_prior_x) 


myobjective = @objective(mymodel, Min, z ) # basic quadratic form

# Solve mymodel
solve(mymodel)

print("note: ignore floating point nits -- e.g. 0.9999999976 should be thought of as 1.0")
print("\n--------------------------------------------------\n")

println("Variable x:Values: ", getvalue(x))

println("Objective value: ", getobjectivevalue(mymodel), "\n")


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

note: ignore floating point nits -- e.g. 0.9999999976 should be thought of as 1.0
--------------------------------------------------
Variable x:Values: [1.0,0.0,1.0,0.0,2.3771984814787784e-9,0.9999999976228016,0.0,1.0,0.0,1.0]
Objective value: 3.923316180950796



In [4]:
for item in x
    if getvalue(item) >0.9
        print(item, getvalue(item), "\n")
    end
end
    

x[1]1.0
x[3]1.0
x[6]0.9999999976228016
x[8]1.0
x[10]1.0


**The next optional step is: convert this to a linear program using some clever transformations.  Warning: This may be hard to follow.**

*To my mind: the LP formulation is much better from a computational standpoint.  However, the essence of the problem is best understood in the algebra associated with the quadratic formulation.*

The idea here is to unpack these quadratic terms into linear combinations and then come up with a very clever constraint at the end to ensure you still get the same result.  (In effect the flourish at the end is to find a hidden probability distribution, so that when you sum along a given axis of some square, you find a 'hidden' probability distribution, and when you sum over the entire valid pmf, you know its sum must be equal to one.  This is reminiscent of my approach to deriving mean and variance for the Poisson distribution.)

So again, for quadratic programming purposes, we have:

$ = z^T C_{3} y + y^T C_{2} x  + x^T C_{1} w + w^T C_{0}v + v^T NominalStartVec$

We can take the above and decompose it as follows:

$=  h^T \big(C_{3} \circ (zy^T) \big) h + h^T \big( C_{2} \circ  (yx^T) \big)h  + h^T \big( C_{1} \circ (xw^T) \big)h + h^T \big(  C_{0} \circ (wv^T) \big)h +  h^T \big(NominalStartVec \circ v \big) $

Notice that $\circ$ denotes the Hadamard product. The above decomposition is quite useful, though perhaps not something seen all that often. 

In this world, our new linear constraint would be to make sure that each of the square matrices that $C_i$ is being (hadamard) multiplied by, sums to one.  The above equation should be interpretted as our implicit *objective function*.  This post will shortly 'peek inside' the hadamard products, and consider summing across the columns or rows of something like $yx^T$.  To be certain, we are not using some sort of legal Linear Algebra manipulation of the above equation.  The 'peek inside' operations are oriented towards our *constraints*, not our *objective function*.  

*To be clear*: The way we leave the quadratic domain for our objective function is by no longer explicitly using $u, v, x, y, z$, but instead using $h^T \big( C_{i} \circ  SomeSquare \big)h$ and modelling out all of those SomeSquare's.  (In the code I stack all of these SomeSquare matrices on top of each other in something I call an x_tensor so you really have $SomeSquare_k$ for k = 0, 1, 2, ... last_observation.  But I don't want to confuse people too much with different terminologies and notation here, so don't dwell on how the collection is organized.)  

The idea though is we no longer explicitly model out these vectors $u, v, x, y, z$ and instead we model out SomeSquare's.  However, we still need to *think* about the implict  $u, v, x, y, z$ because from a mathematical standpoint they still implicitly exist in the various SomeSquare terms.  (Note I omit discussion of how to deal with the one oddball term on the right -- $h^T \big(NominalStartVec \circ v \big)$ -- because once you understand how to deal with the the SomeSquare's, it becomes obvious on how to deal with that piece.  The analogy is just figuring out how to kickstart a recurrence.)  


*onward to the constraints*

So for some term: 

$h^T \big( C_{i} \circ  SomeSquare \big)h$

there is a constraint
$sum(SomeSquare) = 1$

or equivalently

$h^T \big(SomeSquare \big) h = 1$

A subtle but important point is that this is equivalent to part of what we had for our quadratic program.  That is if you look at, for example: 

$h^T \big( C_{2} \circ  (yx^T) \big)h$

where, again $yx^T$ only exists implicitly-- if we insist that 

$h^T (yx^T)h = 1$

This is equivalent to 

$h^T y  = 1$ *and* $x^Th = 1$  We'll return to this point shortly.  

The thing we are mising in our current linear program is: we need to enforce the daisy chaining of the quadratic program that made sure vectors were consistent across terms in the series.  For example, we need to make sure that 

when we look at $z^T C_{3} y$,

and we enforce that 

$h^T (zy^T)h = 1$

which implies 

$h^T z  = 1$ *and* $y^Th = 1$

**AND we need to be sure that we are using the exact same implicit y vector for the SomeSquare associated with** $zy^T$ **as we did with the SomeSquare associated with **$yx^T$.  That is the subject of the following discussion.  

- - - - 
Now for the most challenging / subtle part of this writeup: re to enforce the implicit daisy chain:  

The idea is to play around with various 'outer products' between vectors, recognize them implicitly in our series, and see how we can recover the implicity underlying vectors across terms.  

Key idea: Let's look at this term in the series.  

$h^T \big( C_{2} \circ  (yx^T) \big)h  = sum\big( C_{2} \circ  (yx^T) \big) = sum\big( C_{2} \circ  (ParticularSomeSquare) \big)$ 


What is going on inside the sum?   $C_{2}$ is given to us, and not that interesting.  What we need is to look at the implicit vectors underlying $ParticularSomeSquare$.  

So we examine the implicit outer product here:

$
yx^{T} = \begin{bmatrix}
y_{0}x_{0} &y_{0}x_{1}  &y_{0}x_{2} \\ 
y_{1}x_{0} &y_{1}x_{1}  &y_{1}x_{2}\\ 
y_{2}x_{0} &y_{2}x_{1}  &y_{2}x_{2}
\end{bmatrix}
$

Now that we've looked under the hood, recall the goal is to find an underlying constraint here. If we wanted to sum up the elements by items per column, we *could* think of it as

$
h^T (yx^{T}) = \begin{bmatrix}
noise*x_{0} & noise*x_{1}  &noise*x_{2} \\ 
\end{bmatrix}
$

The reader may be able to guess, the noise, but there is an even easier approach: use associativity.

$
(h^T y) x^{T} = 1*\begin{bmatrix}
x_{0} & x_{1}  &x_{2} \\ 
\end{bmatrix}
$


That is we've 'pinned down' the $x_0$, $x_1$, $x_2$ terms, multiplied by a scalar of 1.  Why is the scalar 1? A few paragraphs ago we noted: $h^T y = 1$


Now, if instead we wanted to sum up the elements by items per row, we *could* think of it as 

$(yx^{T})h $

or, using associativity: 
 $(yx^{T})h = y(x^T h) = \begin{bmatrix}
y_{0}\\ 
y_{1}\\ 
y_{2}
\end{bmatrix} * 1
$

again, a few paragraphs up we noted $x^T h = 1$


**The big idea** 

Left or right multiplying our SomeSquare by the ones vector allows us to isolate the underlying implicit vector of our choosing.  The below flushes this out in some more detail.  
- - - -
*begin extra detail* 

$yx^T$ exists as ParticularSomeSquare and the underlying vectors only implicitly exist.  $yx^T$ has a bunch of zeros in it, and *single* cell equal to 1. This is our goal from a probabilistic modelling standpoint, and we've already enforced this condition.  (If you get lost, think about the fact that the sum of this square must equal 1, and I am allowing the reader to consider this as a binary Integer Program -- this latter piece is technically not needed but it makes the thought experiment a lot easier to follow).  

**claim** if $yx^T$ has single cell = 1, and the rest all zeros, this must be because there is some implicit $y_i^* = 1$  and all other scalar values $y$ are zero -- i.e. $y_j =0$ where $i^* \neq j$ .  AND there must be *one* implicit $x_i = 1$ with all other values in $x$ = 0. 

The proof is easy:  Where all scalars are Boolean in $x$ and $y$. 

we have the constraint: 
$h^T(xy^T)h = 1$

by associativity: 
$(h^Tx) * (y^Th) = 1$

and given that we are dealing with bools, we know that the above holds if and only if $(1)* (1) = 1$

Thus $h^Tx = 1$ and $y^T h = 1$

which, again, given that we are dealing with bools, implies one and only one $y_i = 1$ and one and only one $x_i = 1$

*end extra detail* 
- - - -

Now, with that out of the way, we look at a neighboring term: 

$h^T \big(C_{3} \circ (zy^T) \big) h$

if we 'zoom in'  on $zy^T$ and we summed up items by column, we'd get:

$
h^T (zy^{T}) = (h^T z)y^{T} = 1*y^{T} = \begin{bmatrix}
y_{0} & y_{1}  & y_{2} \\ 
\end{bmatrix}
$



And *this* is the key idea. To be clear, the above manipulations with the h vector are *not* Linear Algebra implications related to the *value* of $h^T \big(C_{3} \circ (zy^T) \big) h$.  

Instead, they are clever ways to tease out a *constraint*.  That is, when we do the above we create a squeeze that allows us to enforce consistency for our implicit $y$ vectors.  Specifically, we have: 

$vec(yx^{T}h) = vec(h^T zy^{T})$

(Small note: In the spirit of Julia: I wrapped a vec() around these to expressions, that way the column vector from $yx^{T}h$ can be treated as a 1 dimensional vector, and the row vector from $h^T zy^{T}$ can also be treated as a 1 dimensional vector.  Note that Julia would ask that you use $.==$ to designate element-wise equality comparisons, but that is a minor implementation detail)  

Put differently, by enforcing $vec(yx^{T}h) = vec(h^T zy^{T})$

we make sure implicit $y_0$ on the left = implicit $y_0$ on the right, *and* that implicit $y_1$ on the left = implicit $y_1$ on the right *and* implicit $y_2$ on the left = implicit $y_2$ on the right

And we can use concept for all adjacent terms in our series to re-write our constraints as linear in nature.  That is, repeatedly using the above constraint between adjacent terms in our series enforces the 'daisy chaining' that occurred in the quadratic objective function. (We just need a touch of special handling to 'kickstart' the process with the far right term $h^T \big(NominalStartVec \circ v \big)$.)

**Near final note:**

In effect the LP form of this project has all linear constraints, and a linear objective function but has $O(m^2 k)$ constraints / variables to worry about, whereas the quadratic form has $O(m k)$ constraints / variables to worry about, but has a quadratic objective function.  

This is a good trade in all cases known to your author, given how heavily optimized and interpretable Linear Program solvers are -- both in terms of purely mechanical issues plus the powerful heuristics that augment them.

There is a particular kicker here: in your author's view, the LP formulation is *much* easier to properly 'sparsify' for HMM's like in our project.  (Technically this should be viewed as a further optimization that has not yet been implemented in the accompanying code, though.)  On top of that the LP program has a 'switch' that can be flipped to officially make the variables involved into (binary) integers if the reader so desires  -- *some* solvers like Gurobi will do mixed integer quadratic programming but this is even more exotic than linear mixed integer programming. 

**Final note: re: Mixed Integer Programs vs Linear Programs**

This posting used the idea of our indicator variables being $\in \{0,1\}$ and $\in [0,1]$ interchangeably because it was expedient to do so.  

In general, running a Mixed Integer Program will yield materially different results than a Linear Program (though there is a whole area of graph / matrix theory devoted to when they will not differ).  This post's approach to this possible divergence is quite simple: look to the underlying problem.  If the underlying problem has one unique Viterbi best solution, then in principle a correct binary Integer Program folmulation will solve for it.  If you relax said Integer Program to become a Linear Program $\in [0,1]$, said program *must* solve for the same solution.  

A much too short sketch of the proof uses the following exchange argument.  The idea is pick some $ 0 < \epsilon \leq 1$ (it can have standard and non-standard parts if you like, though non-standard parts are 'rounded off' in the end -- both in standard analysis and even further with floating point precision).  Assume the Integer Program and/or Viterbi code has an optimal solution $S$, that solves for implicit $Path_i = 1$ and all 'competitors' = 0.  The Linear Program's optimal solution $S^*$ solves for $Path_i = 1 - \epsilon$ and some $Path^* = \epsilon$.  Yet this raises a contradiction -- because there is assumed to be one single Viterbi optimal path, $S^*$ can be improved as $\epsilon \to 0$.  

Put differently there is a contradiction because $S$ is better than $S^*$ -- and Linear Programs can solve for everything Integer Programs can (and more). So the above example suggests that a Linear Program had an optimal feasible solution and instead 'selected' something worse -- *that* is not something that happens.  

- - - -
Note: the LP derivation I came up with -- I've learned -- is almost identical to something called a Network Optimization or Network Flow problem.  Put differently, with a minor tweak, my LP formulation *is* a network optimization problem.  In addition to having extremely fast specialized LP solvers associated with them, Network Programs naturally have integer solutions associated with them.  This is probably the easiest way to think about the integer solution that the above approach derives.
- - - -

Now if the Viterbi optimal 'score' does not have a single unique path, the LP version may give you a 'mixed strategy'.  The idea here is that you will still have a useful optimal score to check against your Viterbi optimal score (assuming you use the same scaling factors in both problems) and they must match. Having a uniquely derived optimal score *was* the point of this whole approach, afterall.  Stepping back a bit: recall that the whole point of this very involved writeup was to come up with a useful way to check your Viterbi algorithm's results.  And if you find a mismatch in optimal scores -- you know that at least one of the algorithms has an incorrect result.  


(If for some reason you *still* are unhappy with the LP returning a mixed strategy in the case of a non-unique optimal path, there are various things that you could do -- the laziest of course would be to just 'flip the switch' and make it a binary Integer Program.)  

After all is said and done the LP approach to this problem is quite interesting.  However it only really exists to check results for Viterbi.  For an even faster way to check sparse Viterbi results, consider an optimized graph search algorithm -- especially Dijkstra for sparse graphs.  

What is interesting about both the QP and LP formulations, however, is that one doesn't need to understand or the specifics of the Viterbi algorithm-- by just following the 'physical' structure of the hidden markov model and the associated linear algebra, you can uncover quite powerful mathematical programming tools to solve the problem for you.  This is a complementary way to view the underlying problem, and can yield more general insights that extend to aparently unrelated domains-- e.g. when analyzing / optimizing the number of potential connection of a neural network with $x$ hidden nodes and a flexibile amount of hidden layers.

In [5]:

function LP_objective_function(likelihood_matrix, A, y_vec, x_tensor, nominal_prior_x, v)
    """
    This is the Linear Objective function for the LP version of checking Viterbi
    
    To cut to the chase, this approach this approach employs something of a hack as it
    perturbs A ever so slightly before taking a logarithm
    
    Setting this up as a 'proper' sparse optimization, without mangling the essense
    of what is going on, is quite manageable but left as a subsequent optimization.  
    
    i.e. doing so would improve computational speed, 
    but add complications without adding improving insight
    
    """
    epsilon = 0.000000000000000001
        
    assert(length(size(y_vec)) == 1) # a proper vector here    
    entries_in_y = size(y_vec)[1] # then we grab its dimension here
    
    m, n = size(x_tensor[:,:,1])
    assert(m == n)  
    # i.e. the tensor should be composed of square matrices stacked on top of each other
    
    collector = dot(nominal_prior_x, v) 
    # nominal_prior_x is ALREADY in - log space

    for idx = 1:entries_in_y        
        # the_diagonal_piece = diagm(likelihood_matrix[:,y_vec[idx]]) 
        # not needed if we use broadcasting
        if idx > 1 
            adj_A = (likelihood_matrix[:,y_vec[idx]] .* A) + epsilon
            # below is the correct linear algebra setup, but above is the broadcasting equivalent
            # adj_A = *(the_diagonal_piece, A) + epsilon            
        else 
            # again special handling for the first instance is required in Viterbi coin example
            # in coin tossing example we start with a coin toss 
            # i.e. there is no matrix A product 'just before dawn'
            # so if idx == 1, there is just a likelihood function, no matrix A included
            the_diagonal_piece = diagm(likelihood_matrix[:,y_vec[idx]])
            adj_A = the_diagonal_piece + epsilon
        end        
        adj_A = -log(adj_A)
        # recall this log transformation process is being -- inefficiently done, over and over 
        # clearly there are optimization opportunities available here, 
        # though they are outside the scope of what the purpose of this code / writeup is
       
#         collector += sum(adj_A .* x_tensor[:,:,idx]) # grab slice off the top and do hadamard
        # the above line is correct but perhaps too pythonic.  Julia asks that we use the below:
        append!(collector, sum(adj_A .* x_tensor[:,:,idx]))# grab slice off the top and do hadamard
        # specifically, if you use the += for a LARGE program, you'll get the below warning:
#         WARNING: The addition operator has been used on JuMP expressions a large number of times. 
#         This warning is safe to ignore but may indicate that model generation is slower than 
#         necessary. For performance reasons, you should not add expressions in a loop. 
#         Instead of x += y, use append!(x,y) to modify x in place. If y is a single variable, 
#         you may also use push!(x, coef, y) in place of x += coef*y.
        
    end
    return collector
end


LP_objective_function (generic function with 1 method)

In [6]:
using JuMP

using Cbc


function do_the_LP(y_vec)

    mymodel = Model(solver= CbcSolver())    
    # For the avoidance of doubt:
    # A is the transition matrix, and L is a collection of likelihood vectors

    A = [3/4  1/4; 
         1/4  3/4]

    m, n = size(A)

    assert(m == n)


    L = [1/4 1/2  1/2; 
         1/8 1/4  3/4]

    # y_vec = [2, 3, 3, 3]    
    nominal_prior_x = [1/4, 1/8]

    nominal_prior_x = nominal_prior_x / sum(nominal_prior_x)
    nominal_prior_x = -log(nominal_prior_x)
    # convert to logspace before beginning 

    k_depth = size(y_vec)[1]

    # see comments
    # integer programming setup is shown below, if the user is so interested
    # @variable(mymodel, x[i = 1: m*n*k_depth], Bin)
    # @variable(mymodel, v[i = 1: m], Bin) 

    # comment out the below lines for x,v and uncomment the above two lines, 
    # to make this into a formal integer program

    @variable(mymodel, 0<=x[i = 1: m*n*k_depth]<=1)
    @variable(mymodel, 0<=v[i = 1: m]<=1) 
    
    

    # v kickstarts this process -- it relates to the prior, doesn't need its own 'square'
    # as such I keep it separate from x 
    # because x is a sequence of square matrices stacked on top of each other (in tensor)

    x_tensor = reshape(x, (m, n, k_depth))

    ones_vc = ones(m,1) # this is a column ones vector


    # All constraints are actually Linear Equalities
    mini_kickstart_constraint = @constraint(mymodel, sum(v) == 1) 
    # similar to my_constraint, but just used to kickstart this process

    my_constraint = @constraint(mymodel, [kdx = 1: k_depth], sum(x_tensor[:,:,kdx]) == 1)
    # in some sense you don't actually need this constraint, and I removed it 
    # to make the model seem more 'network flow like'... but it turns out things go a lot faster with it in
    # specifically I ran a case where my_y_vec = rand(1:3, 100000), and it randomly made a vector
    # then I solved the problem with this constraint commented out and it took 834.095970 seconds
    # then I solved the problem with this constraint in, and it took 40.075415 seconds
    # the difference is kind of shocking
    
    # in effect: you must select one cell in each of the square matrices -- 
    # and all other values are zero

    kickstart_constraint = @constraint(mymodel, vec(*(transpose(ones_vc), x_tensor[:,:,1])) - v .== 0)

    # the squeeze of implicilty identical vectors, is shown below
    for kdx = 2: k_depth
        @constraint(mymodel, vec(*(x_tensor[:,:, (kdx - 1)], ones_vc)) - vec(*(transpose(ones_vc), x_tensor[:,:, kdx]) ) .== 0)
    end  
    # this is the trickiest constraint, and what is needed to convert this from quadratic to linear
    # it exploits a 'hidden pmf' embodded in each implicit vector 
    # associated with a given level (kdx) of the x_tensor

    # dummy constraint at the very end, maybe makes it a network problem? 
    @constraint(mymodel, - sum(x_tensor[:,:, k_depth])  == -1)
    
    my_z = LP_objective_function(L, A, y_vec, x_tensor, nominal_prior_x, v) # a scalar result

    # if you know the score of the best case, the simplest way to get the second best score 
    # (i.e. second best > best) is: make variables binary integers
    # then uncomment out the below two lines
    # user_inputs_best_score = 3.92331 + 0.001
    # second_best_constraint = @constraint(mymodel, my_z >= user_inputs_best_score) 
    # unlike the above constraints this is a linear inequality, however.  

    myobjective = @objective(mymodel, Min, my_z ) # basic linear form
    # Solve mymodel
    solve(mymodel)

    # println("Variable x:Values: ", getvalue(x))

    println("Objective value: ", getobjectivevalue(mymodel))
    return k_depth, v, x_tensor, ones_vc
end

do_the_LP (generic function with 1 method)

In [7]:
my_y_vec = [2, 3, 3, 3];

# my_y_vec = rand(1:3, 100000);
# if you want to try running the LP on a much larger case
# uncomment out the above line, and run it 

k_depth, v, x_tensor, ones_vc = do_the_LP(my_y_vec);

Objective value: 3.9233170120469048


In [8]:
print("The below prints out an interpretable form of the results \n")
print(getvalue(v))
for k = 1:k_depth
    result = vec(*(x_tensor[:,:,k], ones_vc))
    print("\n", getvalue(result))
end

print("\n\nnote due to floating point issues, a value like 0.999999, it should be interpretted as 1.0")    

The below prints out an interpretable form of the results 
[1.0,0.0]
[1.0,0.0]
[0.0,1.0]
[0.0,1.0]
[0.0,1.0]

note due to floating point issues, a value like 0.999999, it should be interpretted as 1.0